In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

import warnings
def ignore_warn(*args, **kwargs):
    pass                                                      
warnings.warn = ignore_warn 

train = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
it_cat = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv')


Bad key "text.kerning_factor" on line 4 in
C:\Users\gherrera\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


## Data Cleaning

In [2]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [3]:
#Discovered in discussions
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [4]:
#train.isnull().sum()

In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2935849.000,2935849.000,2935849.000,2935849.000,2935849.000
mean,14.570,33.311,10197.227,890.853,1.243
std,9.423,16.137,6324.297,1729.800,2.619
min,0.000,2.000,0.000,-1.000,-22.000
25%,7.000,22.000,4476.000,249.000,1.000
50%,14.000,31.000,9343.000,399.000,1.000
75%,23.000,48.000,15684.000,999.000,1.000
max,33.000,59.000,22169.000,307980.000,2169.000


In [6]:
train = train[train['item_cnt_day'] < 1000]
train = train[train['item_price'] < 100000]

In [7]:
train[train.item_price < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,15.05.2013,4,32,2973,-1.000,1.000


In [8]:
train = train.sort_values(by=['date','date_block_num','shop_id','item_id'], ascending = True)

In [9]:
train['item_price'] = train['item_price'].replace(-1,np.nan)

In [10]:
train[(train.shop_id == 32) & (train.item_id == 2973)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
187844,05.02.2013,1,32,2973,2499.000,1.000
555026,05.06.2013,5,32,2973,1249.500,1.000
484684,06.05.2013,4,32,2973,2499.000,1.000
187845,14.02.2013,1,32,2973,2499.000,1.000
67433,15.01.2013,0,32,2973,2499.000,3.000
484683,15.05.2013,4,32,2973,nan,1.000
67432,17.01.2013,0,32,2973,2499.000,1.000
67431,18.01.2013,0,32,2973,2499.000,1.000
67430,21.01.2013,0,32,2973,2499.000,1.000
67429,22.01.2013,0,32,2973,2499.000,1.000


In [11]:
train.loc[484683,'item_price'] = 2499

In [12]:
#plt.boxplot(train['item_cnt_day'])
#plt.boxplot(train['item_price'])

In [13]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
32569,01.01.2013,0,2,991,99.000,1.000
32552,01.01.2013,0,2,1472,2599.000,1.000
32632,01.01.2013,0,2,1905,249.000,1.000
32459,01.01.2013,0,2,2920,599.000,2.000
32504,01.01.2013,0,2,3320,1999.000,1.000


## Feature Engineering

In [14]:
train.shape[0]

2935846

In [15]:
#DataFrame with last price of itmes per shop and per month (will be used to assign price to items after unstacking)
sim_prices = train[['date_block_num','shop_id','item_id','item_price']].drop_duplicates(['date_block_num','shop_id','item_id'],keep='last').reset_index(drop=True)

In [16]:
#Concatenating datasets
n_train = len(train)
n_test = len(test)
test_id = test['ID']
test['date_block_num'] = 34
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop('ID', axis=1, inplace = True)

In [17]:
print('test: ', test.shape[0],'\ntrain: ', train.shape[0], '\nall: ', all_data.shape[0])

test:  214200 
train:  2935846 
all:  3150046


In [18]:
from datetime import datetime
all_data['date'] = pd.to_datetime(all_data['date'], format="%d.%m.%Y")
all_data['date'] = pd.to_datetime(all_data['date'], format="%d.%m.%Y")
#all_data['weekday'] = all_data['date'].dt.weekday
all_data['monthyear'] = pd.DatetimeIndex(all_data['date']).month

In [19]:
#all_data[all_data.date_block_num == 33]
all_data.loc[all_data.date_block_num == 34,'monthyear'] = 11

In [20]:
ms = pd.Series(all_data['date_block_num'].unique())
wkn_df = pd.DataFrame()
wknds = []
for m in ms:
    temp = all_data[all_data['date_block_num'] == m]
    wds = pd.Series(temp['date'].unique()).dt.weekday
    n_wknd_days = 0
    for wd in wds:
        if wd in [5,6]:
            n_wknd_days += 1
    wknds.append(n_wknd_days)
    
wkn_df['date_block_num'] = ms
wkn_df['wknd_days'] = wknds
all_data = pd.merge(all_data, wkn_df, how='left', left_on='date_block_num', right_on='date_block_num')
all_data.loc[all_data.date_block_num == 34, 'wknd_days'] = 9

In [21]:
all_data.shape[0]

3150046

In [22]:
#grouping
all_gr = all_data.groupby(['date_block_num','shop_id','item_id','monthyear','wknd_days']).agg({'item_price': 'mean', 'item_cnt_day': 'sum'}).reset_index()

In [23]:
dcols = {'item_cnt_day':'item_cnt_month'}
all_gr.rename(columns = dcols, inplace=True)
all_gr.head()

,date_block_num,shop_id,item_id,monthyear,wknd_days,item_price,item_cnt_month
0,0,2,27,1.000,8,2499.000,1.000
1,0,2,33,1.000,8,499.000,1.000
2,0,2,317,1.000,8,299.000,1.000
3,0,2,438,1.000,8,299.000,1.000
4,0,2,471,1.000,8,399.000,2.000


In [24]:
all_gr.shape[0]

1823322

In [25]:
all_gr[all_gr.duplicated()]

,date_block_num,shop_id,item_id,monthyear,wknd_days,item_price,item_cnt_month


In [26]:
all_gr.head()

,date_block_num,shop_id,item_id,monthyear,wknd_days,item_price,item_cnt_month
0,0,2,27,1.000,8,2499.000,1.000
1,0,2,33,1.000,8,499.000,1.000
2,0,2,317,1.000,8,299.000,1.000
3,0,2,438,1.000,8,299.000,1.000
4,0,2,471,1.000,8,399.000,2.000


In [27]:
all_gr.isnull().sum()

date_block_num         0
shop_id                0
item_id                0
monthyear              0
wknd_days              0
item_price        214200
item_cnt_month         0
dtype: int64

### assign price to test set

#### First, last price of same item in same shop

In [28]:
assign_price = all_gr.groupby(['shop_id','item_id','item_price']).agg({'date_block_num': max}).reset_index()

In [29]:
assign_price = assign_price.sort_values('date_block_num').drop_duplicates(['shop_id','item_id'],keep='last')

In [30]:
assign_price.columns = ['shop_id','item_id','ass_price','date_block_num']
all_gr = pd.merge(all_gr, assign_price[['shop_id','item_id','ass_price']], how='left', on=['shop_id','item_id'])

In [31]:
all_gr.shape[0]

1823322

In [32]:
all_gr['item_price'] = all_gr['item_price'].fillna(all_gr['ass_price'])
#all_gr['item_price']= np.where(all_gr['item_price'].isnull(), all_gr['ass_price'], all_gr['item_price'])
all_gr.drop('ass_price',axis=1,inplace=True)

In [33]:
all_gr.shape[0]

1823322

In [34]:
#all_gr.isnull().sum()

#### If not available, last price of same item in any shop

In [35]:
assign_price_item = all_gr.groupby(['item_id','item_price']).agg({'date_block_num': max}).reset_index()
assign_price_item = assign_price_item.sort_values('date_block_num').drop_duplicates(['item_id'],keep='last')

In [36]:
assign_price_item.columns = ['item_id','ass_price','date_block_num']
all_gr = pd.merge(all_gr, assign_price_item[['item_id','ass_price']], how='left', on='item_id')

In [37]:
all_gr['item_price'] = all_gr['item_price'].fillna(all_gr['ass_price'])
all_gr.drop('ass_price',axis=1,inplace=True)

In [38]:
all_gr.shape[0]

1823322

In [39]:
#all_gr.isnull().sum()

#### If not available, median price of item category

In [40]:
#Assign median price of category
#get category column
all_gr = pd.merge(all_gr, items[['item_id','item_category_id']], on='item_id')
all_gr.shape[0]

1823322

In [41]:
assign_price_cat = all_gr.groupby(['item_category_id','date_block_num']).agg({'item_price': 'median'}).reset_index()
assign_price_cat = assign_price_cat.sort_values('date_block_num').drop_duplicates(['item_category_id'],keep='last')

In [42]:
assign_price_cat.columns = ['item_category_id','date_block_num','ass_price']
all_gr = pd.merge(all_gr, assign_price_cat[['item_category_id','ass_price']], how='left', on='item_category_id')

In [43]:
all_gr['item_price'] = all_gr['item_price'].fillna(all_gr['ass_price'])
all_gr.drop('ass_price',axis=1,inplace=True)

In [44]:
all_gr.shape[0]

1823322

In [45]:
#all_gr.isnull().sum()

#### If not available, median price of items of the shop

In [46]:
#assign median price of shop
assign_price_shop = all_gr.groupby(['shop_id','date_block_num']).agg({'item_price': 'median'}).reset_index()
assign_price_shop = assign_price_shop.sort_values('date_block_num').drop_duplicates(['shop_id'],keep='last')

assign_price_shop.columns = ['shop_id','date_block_num','ass_price']
all_gr = pd.merge(all_gr, assign_price_shop[['shop_id','ass_price']], how='left', on='shop_id')

all_gr['item_price'] = all_gr['item_price'].fillna(all_gr['ass_price'])
all_gr.drop('ass_price',axis=1,inplace=True)

all_gr.shape[0]

1823322

In [47]:
all_gr.isnull().sum()

date_block_num      0
shop_id             0
item_id             0
monthyear           0
wknd_days           0
item_price          0
item_cnt_month      0
item_category_id    0
dtype: int64

### generate rows for each shop and item for each month

In [48]:
all_gr.head()

,date_block_num,shop_id,item_id,monthyear,wknd_days,item_price,item_cnt_month,item_category_id
0,0,2,27,1.000,8,2499.000,1.000,19
1,0,11,27,1.000,8,1890.000,1.000,19
2,0,19,27,1.000,8,2499.000,1.000,19
3,0,26,27,1.000,8,2499.000,1.000,19
4,0,31,27,1.000,8,2499.000,1.000,19


In [49]:
X_all=all_gr.set_index(['date_block_num','shop_id','item_id']).unstack('date_block_num').stack(dropna=False).reset_index().fillna(0)

In [50]:
X_all.head()

,shop_id,item_id,date_block_num,monthyear,wknd_days,item_price,item_cnt_month,item_category_id
0,2,27,0,1.000,8.000,2499.000,1.000,19.000
1,2,27,1,0.000,0.000,0.000,0.000,0.000
2,2,27,2,0.000,0.000,0.000,0.000,0.000
3,2,27,3,0.000,0.000,0.000,0.000,0.000
4,2,27,4,0.000,0.000,0.000,0.000,0.000


In [51]:
X_all.shape[0]

18257295

#### fixing month and n of weekend days

In [52]:
dt_ = all_gr[['date_block_num','monthyear','wknd_days']].drop_duplicates()

In [53]:
X_all = pd.merge(X_all[['shop_id','item_id','date_block_num','item_price','item_cnt_month','item_category_id']],dt_, how='left',on='date_block_num')

In [54]:
X_all.shape[0]

18257295

#### fixing item category

In [55]:
X_all.drop('item_category_id', axis = 1, inplace = True)
X_all = pd.merge(X_all, items[['item_id','item_category_id']], on='item_id')

In [56]:
X_all.shape[0]

18257295

#### fixing item price

In [57]:
X_all.head()

,shop_id,item_id,date_block_num,item_price,item_cnt_month,monthyear,wknd_days,item_category_id
0,2,27,0,2499.000,1.000,1.000,8,19
1,2,27,1,0.000,0.000,2.000,8,19
2,2,27,2,0.000,0.000,3.000,10,19
3,2,27,3,0.000,0.000,4.000,8,19
4,2,27,4,0.000,0.000,5.000,8,19


In [58]:
# I will use fillna method until I find an efficient way to assign price using sim_prices. Some items will get assigned the price incorrectly, as this following method assigns previous value if nan

In [59]:
X_all['item_price'] = np.where(X_all['date_block_num'] == 0, X_all['item_price'],X_all['item_price'].replace(0,np.nan))

In [60]:
#rows with no item price at the start have value 0 instead of nan, so all prices with 0 will need to be treated 
#this way we avoid wrong assignment with ffill method
X_all['assigned_price'] = X_all['item_price'].fillna(method='ffill')

In [61]:
X_all.head()

,shop_id,item_id,date_block_num,item_price,item_cnt_month,monthyear,wknd_days,item_category_id,assigned_price
0,2,27,0,2499.000,1.000,1.000,8,19,2499.000
1,2,27,1,nan,0.000,2.000,8,19,2499.000
2,2,27,2,nan,0.000,3.000,10,19,2499.000
3,2,27,3,nan,0.000,4.000,8,19,2499.000
4,2,27,4,nan,0.000,5.000,8,19,2499.000


In [62]:
#this part is not needed anymore, since we didn't replace first month with nans
#X_all['fprice'] = X_all.apply(lambda row: row['item_price'] if row['date_block_num'] == 0 else row['assigned_price'], axis=1)

In [63]:
print('prices with value 0 to be solved:')
print(X_all[X_all.assigned_price == 0].shape[0])

prices with value 0 to be solved:
7951529


In [64]:
X_all.shape[0]

18257295

#### filling prices with median

In [65]:
assign_price_item = sim_prices.groupby(['item_id']).agg({'item_price': 'median'}).reset_index()

assign_price_item.columns = ['item_id','ass_price']
X_all = pd.merge(X_all, assign_price_item[['item_id','ass_price']], how='left', on='item_id')

#X_all['assigned_price'] = X_all['assigned_price'].replace(0,X_all['ass_price'])
#X_all.drop('ass_price',axis=1,inplace=True)

In [66]:
X_all['fprice'] = X_all.apply(lambda row: row['ass_price'] if row['assigned_price'] == 0 else row['assigned_price'], axis=1)

In [67]:
X_all.shape[0]

18257295

In [68]:
X_all.drop(['item_price','assigned_price','ass_price'], axis = 1, inplace = True)

In [69]:
#X_all.isnull().sum()

#### rest: median of item category

In [70]:
X_all = pd.merge(X_all, assign_price_cat[['item_category_id','ass_price']], how='left', on='item_category_id')

In [71]:
X_all['fprice'] = X_all.fillna(X_all['ass_price'])

In [72]:
X_all.drop(['ass_price'], axis = 1, inplace = True)
X_all.shape[0]

18257295

In [73]:
#X_all.isnull().sum()

In [74]:
#assign median price of shop
X_all = pd.merge(X_all, assign_price_shop[['shop_id','ass_price']], how='left', on='shop_id')

X_all['fprice'] = X_all.fillna(X_all['ass_price'])
X_all.drop(['ass_price'], axis = 1, inplace = True)
X_all.shape[0]

18257295

In [75]:
X_all.isnull().sum()

shop_id             0
item_id             0
date_block_num      0
item_cnt_month      0
monthyear           0
wknd_days           0
item_category_id    0
fprice              0
dtype: int64

In [76]:
#X_all.to_csv('X_all_price.csv')

In [77]:
#X_all = pd.read_csv('X_all_price.csv')

In [78]:
X_all['shop_id'] = X_all['shop_id'].astype(str)
X_all['item_id'] = X_all['item_id'].astype(str)
X_all['shop_item'] = X_all['shop_id']+X_all['item_id']

In [79]:
X_all.tail()

,shop_id,item_id,date_block_num,item_cnt_month,monthyear,wknd_days,item_category_id,fprice,shop_item
18257290,59,18018,30,0.000,7.000,8,57,59.000,5918018
18257291,59,18018,31,0.000,8.000,10,57,59.000,5918018
18257292,59,18018,32,0.000,9.000,8,57,59.000,5918018
18257293,59,18018,33,0.000,10.000,9,57,59.000,5918018
18257294,59,18018,34,0.000,11.000,9,57,59.000,5918018


In [80]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_label'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_label']]

In [81]:
shops['shop_id'] = shops['shop_id'].astype(str)
X_all = pd.merge(X_all,shops, how='left', on='shop_id')

In [82]:
X_all.shape[0]

18257295

In [83]:
X_all.isnull().sum()

shop_id             0
item_id             0
date_block_num      0
item_cnt_month      0
monthyear           0
wknd_days           0
item_category_id    0
fprice              0
shop_item           0
city_label          0
dtype: int64

In [84]:
def label(element):
    if element < 400:
        return 1
    elif element >=400 and element < 1200:
        return 2
    elif element>=1200 and element < 4000:
        return 3
    else:
        return 4

X_all['price_range'] = X_all['fprice'].apply(label)

#### Lagged values

In [85]:
#clip target
X_all['item_cnt_month'] = X_all['item_cnt_month'].clip(0,20)

In [86]:
def lag(df, n, col):
    temp = df[['date_block_num','shop_id','item_id',col]]
    for i in n:
        temp.columns = ['date_block_num','shop_id','item_id','prev_' + str(i) + '_' + col]
        temp['date_block_num'] += i
        df = pd.merge(df, temp, how='left', left_on=['shop_id','item_id','date_block_num'], right_on=['shop_id','item_id','date_block_num'])
    return df

X_all = lag(X_all, [1,2,3,6,12], 'item_cnt_month')

In [87]:
X_all.shape[0]

18257295

In [88]:
#total avg month
temp = X_all.copy()
temp = temp.groupby('date_block_num').agg({'item_cnt_month':'mean'})
temp.columns = ['month_avg_item_cnt']
X_all = pd.merge(X_all,temp,how='left',on='date_block_num')
X_all = lag(X_all, [1], 'month_avg_item_cnt')
X_all.drop(['month_avg_item_cnt'], axis=1, inplace=True)

In [89]:
#total avg per item
temp = X_all.copy()
temp = temp.groupby(['date_block_num','item_id']).agg({'item_cnt_month':'mean'})
temp.columns = ['month_item_avg_item_cnt']
X_all = pd.merge(X_all,temp,how='left',on=['date_block_num','item_id'])
X_all = lag(X_all, [1,3,6,12], 'month_item_avg_item_cnt')
X_all.drop(['month_item_avg_item_cnt'], axis=1, inplace=True)

In [90]:
#total avg per shop
temp = X_all.copy()
temp = temp.groupby(['date_block_num','shop_id']).agg({'item_cnt_month':'mean'})
temp.columns = ['month_shop_avg_item_cnt']
X_all = pd.merge(X_all,temp,how='left',on=['date_block_num','shop_id'])
X_all = lag(X_all, [1,3,6,12], 'month_shop_avg_item_cnt')
X_all.drop(['month_shop_avg_item_cnt'], axis=1, inplace=True)

In [91]:
#average per category
temp = X_all.copy()
temp = temp.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month':'mean'})
temp.columns = ['month_cat_avg_item_cnt']
X_all = pd.merge(X_all,temp,how='left',on=['date_block_num','item_category_id'])
X_all = lag(X_all, [1,3,6,12], 'month_cat_avg_item_cnt')
X_all.drop(['month_cat_avg_item_cnt'], axis=1, inplace=True)

In [92]:
#average per city
temp = X_all.copy()
temp = temp.groupby(['date_block_num', 'item_id', 'city_label']).agg({'item_cnt_month': 'mean'})
temp.columns = [ 'date_item_city_avg_item_cnt' ]

X_all = pd.merge(X_all, temp, how='left', on=['date_block_num', 'item_id', 'city_label'])
X_all = lag(X_all, [1], 'date_item_city_avg_item_cnt')
X_all.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)

In [93]:
#trend
X_all['cnt_change'] = (X_all['item_cnt_month']/(X_all['prev_1_item_cnt_month'])) - 1
X_all['cnt_change'] = X_all['cnt_change'].fillna(0).replace(np.inf,1)

In [94]:
X_all = X_all.fillna(0)

In [95]:
#Check best features
X_all.corr()['item_cnt_month'].sort_values()

item_category_id                     -0.050
date_block_num                       -0.045
prev_12_month_shop_avg_item_cnt      -0.008
monthyear                            -0.004
wknd_days                             0.000
city_label                            0.013
fprice                                0.014
prev_12_month_item_avg_item_cnt       0.021
prev_12_item_cnt_month                0.025
prev_1_month_avg_item_cnt             0.027
prev_6_month_shop_avg_item_cnt        0.031
prev_12_month_cat_avg_item_cnt        0.037
prev_3_month_shop_avg_item_cnt        0.065
prev_1_month_shop_avg_item_cnt        0.097
prev_6_month_item_avg_item_cnt        0.123
prev_6_item_cnt_month                 0.132
prev_6_month_cat_avg_item_cnt         0.134
prev_3_month_cat_avg_item_cnt         0.195
prev_1_month_cat_avg_item_cnt         0.239
prev_3_item_cnt_month                 0.240
prev_3_month_item_avg_item_cnt        0.241
prev_2_item_cnt_month                 0.345
cnt_change                      

In [96]:
X_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18257295 entries, 0 to 18257294
Data columns (total 31 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   shop_id                             object 
 1   item_id                             object 
 2   date_block_num                      int64  
 3   item_cnt_month                      float64
 4   monthyear                           float64
 5   wknd_days                           int64  
 6   item_category_id                    int64  
 7   fprice                              float64
 8   shop_item                           object 
 9   city_label                          int32  
 10  price_range                         int64  
 11  prev_1_item_cnt_month               float64
 12  prev_2_item_cnt_month               float64
 13  prev_3_item_cnt_month               float64
 14  prev_6_item_cnt_month               float64
 15  prev_12_item_cnt_month              float64
 16

## Modelling

In [97]:
import gc
import pickle
X_all.to_pickle('data.pkl')
gc.collect();

In [98]:
data = pd.read_pickle('data.pkl')

In [99]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

X_train[['shop_id','item_id','shop_item']] = X_train[['shop_id','item_id','shop_item']].astype('int64')
X_valid[['shop_id','item_id','shop_item']] = X_valid[['shop_id','item_id','shop_item']].astype('int64')

In [100]:
list(X_train.columns)

['shop_id',
 'item_id',
 'date_block_num',
 'monthyear',
 'wknd_days',
 'item_category_id',
 'fprice',
 'shop_item',
 'city_label',
 'price_range',
 'prev_1_item_cnt_month',
 'prev_2_item_cnt_month',
 'prev_3_item_cnt_month',
 'prev_6_item_cnt_month',
 'prev_12_item_cnt_month',
 'prev_1_month_avg_item_cnt',
 'prev_1_month_item_avg_item_cnt',
 'prev_3_month_item_avg_item_cnt',
 'prev_6_month_item_avg_item_cnt',
 'prev_12_month_item_avg_item_cnt',
 'prev_1_month_shop_avg_item_cnt',
 'prev_3_month_shop_avg_item_cnt',
 'prev_6_month_shop_avg_item_cnt',
 'prev_12_month_shop_avg_item_cnt',
 'prev_1_month_cat_avg_item_cnt',
 'prev_3_month_cat_avg_item_cnt',
 'prev_6_month_cat_avg_item_cnt',
 'prev_12_month_cat_avg_item_cnt',
 'prev_1_date_item_city_avg_item_cnt',
 'cnt_change']

In [101]:
best_feat = ['date_block_num',
 'monthyear',
 'wknd_days',
 'item_category_id',
 #'fprice',
 'shop_item',
 'city_label',
 'price_range',
 'prev_1_item_cnt_month',
 'prev_2_item_cnt_month',
 'prev_3_item_cnt_month',
 'prev_6_item_cnt_month',
 'prev_12_item_cnt_month',
 'prev_1_month_avg_item_cnt',
 'prev_1_month_item_avg_item_cnt',
 'prev_3_month_item_avg_item_cnt',
 'prev_6_month_item_avg_item_cnt',
 'prev_12_month_item_avg_item_cnt',
 'prev_1_month_shop_avg_item_cnt',
 'prev_3_month_shop_avg_item_cnt',
 'prev_6_month_shop_avg_item_cnt',
 'prev_12_month_shop_avg_item_cnt',
 'prev_1_month_cat_avg_item_cnt',
 'prev_3_month_cat_avg_item_cnt',
 'prev_6_month_cat_avg_item_cnt',
 'prev_12_month_cat_avg_item_cnt',
 'cnt_change',
 'prev_1_date_item_city_avg_item_cnt',
 #'si_label']
            ]

In [102]:
X_train = X_train[best_feat]
X_valid = X_valid[best_feat]
X_test = X_test[best_feat]

In [103]:
del data
gc.collect();

### XGBRegressor

In [104]:
from xgboost import XGBRegressor

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 5)

[18:24:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:24:02] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	validation_0-rmse:0.89987	validation_1-rmse:0.839
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.837849	validation_1-rmse:0.782349
[2]	validation_0-rmse:0.788497	validation_1-rmse:0.73869
[3]	validation_0-rmse:0.731029	validation_1-rmse:0.690049
[4]	validation_0-rmse:0.691219	validation_1-rmse:0.651477
[5]	validation_0-rmse:0.67702	validation_1-rmse:0.633759
[6]	validation_0-rmse:0.642421	validation_1-rmse:0.602736
[7]	validation_0-rmse:0.631658	validation_1-rmse:0.588554
[8]	validation_0-rmse:0.604625	validation_1-rmse:0.561839
[9]	val

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=300, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.8, verbosity=1)

In [122]:
data = pd.read_pickle('data.pkl')
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
x = X_test[best_feat]
x['shop_item'] = X_test['shop_item'].astype('int64')

In [123]:
predictions1 = model.predict(x).clip(0, 20)
predictions2 = model.predict(x)

In [128]:
X_test[['shop_id','item_id']] = X_test[['shop_id','item_id']].astype('int64')

X_test['item_cnt_month'] = predictions1
sub = pd.merge(test,X_test, how='left',on=['shop_id','item_id'])
sub['item_cnt_month'] = sub['item_cnt_month'].fillna(0)
sub[['ID','item_cnt_month']].to_csv('sub4.csv', index = False)

In [127]:
X_test['item_cnt_month'] = predictions2
sub = pd.merge(test,X_test, how='left',on=['shop_id','item_id'])
sub['item_cnt_month'] = sub['item_cnt_month'].fillna(predictions2.mean())
sub[['ID','item_cnt_month']].to_csv('sub5.csv', index = False)

### LGBMRegressor

In [3]:
data = pd.read_pickle('data.pkl')
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

X_train[['shop_id','item_id','shop_item']] = X_train[['shop_id','item_id','shop_item']].astype('int64')
X_valid[['shop_id','item_id','shop_item']] = X_valid[['shop_id','item_id','shop_item']].astype('int64')

In [4]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [5]:
model_lgb.fit(X_train,Y_train)
predictions = model_lgb.predict(X_valid)

mse = mean_squared_error(predictions,Y_valid)
mse**(1/2)

0.4519438538632103

In [11]:
f = ['prev_6_month_item_avg_item_cnt'        
,'prev_6_item_cnt_month'                 
,'prev_6_month_cat_avg_item_cnt'         
,'prev_3_month_cat_avg_item_cnt'         
,'prev_1_month_cat_avg_item_cnt'         
,'prev_3_item_cnt_month'                 
,'prev_3_month_item_avg_item_cnt'        
,'prev_2_item_cnt_month'                 
,'cnt_change'                            
,'prev_1_month_item_avg_item_cnt'        
,'prev_1_date_item_city_avg_item_cnt'    
,'prev_1_item_cnt_month'
,'shop_item']   

In [8]:
model_lgb.fit(X_train[f],Y_train)
predictions2 = model_lgb.predict(X_valid[f])

mse = mean_squared_error(predictions2,Y_valid)
mse**(1/2)

0.4728155244292936